<a href="https://colab.research.google.com/github/chineidu/NLP-Tutorial/blob/main/notebook/03_tensorflow/03_classification_with_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification With TensorFlow

Types of Classification
1. Binary
2. Multiclass
3. Multilabel


In [1]:
# Standard imports
from typing import List, Tuple, Dict, Optional
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

### Create Data For Classification

* Using scikit-learn

In [2]:
from sklearn.datasets import make_circles

RANDOM_STATE = 1
SEED = 2
NOISE = 0.05
N_SAMPLES = 2_000

X, y = make_circles(N_SAMPLES, noise=NOISE, random_state=RANDOM_STATE)

X.shape, y. shape

((2000, 2), (2000,))